In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Step 1: Load and Quantize the Model
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [2]:
# Quantize the model weights for efficient memory usage
model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

In [3]:
# Step 2: Define Low-Rank Adaptation Matrices Manually
# Assuming the model layer has 2048 input and output features
# Define low-rank matrices A and B (rank = 4 for this example)
rank = 4
in_features = 2048
out_features = 2048

# Low-rank matrices for adaptation (A: 2048x4, B: 4x2048)
A = torch.randn(in_features, rank)  # Matrix A with shape (2048, 4)
B = torch.randn(rank, out_features) # Matrix B with shape (4, 2048)


In [4]:
# Define a function for applying the LoRA transformation
def apply_lora(x):
    # Perform low-rank adaptation by applying A and B
    lora_output = torch.matmul(torch.matmul(x, A), B)  # shape remains (batch, seq_len, 2048)
    return lora_output

# Step 3: Define the Forward Pass with LoRA
prompt = "The cat sat on"
inputs = tokenizer(prompt, return_tensors="pt")

# Perform forward pass and inject LoRA adjustments
with torch.no_grad():  # fine-tuning, so we do not require gradient calculations
    # Step 3a: Get the embeddings for the input
    embeddings = model.model.embed_tokens(inputs["input_ids"])

    # Step 3b: Process through the first attention layer of LLaMA with LoRA
    attention_layer = model.model.layers[0].self_attn  # Get first attention layer

    # Pass embeddings through q_proj layer without transposing
    q_proj_output = attention_layer.q_proj(embeddings)  # Should have shape (batch, seq_len, 2048)

    # Apply LoRA adjustment
    lora_adjustment = apply_lora(embeddings)  # This should be (batch, seq_len, 2048)
    attention_output = q_proj_output + lora_adjustment  # Combine with LoRA output

    # Step 4: Pass through LM Head for final output logits
    logits = model.lm_head(attention_output)


In [5]:

# Step 4: Obtain Predictions
# Apply softmax to get probabilities for the next token in the vocabulary
probs = torch.softmax(logits, dim=-1)
predicted_token = torch.argmax(probs, dim=-1)

# Decode the predicted token ID to get the word
predicted_word = tokenizer.decode(predicted_token[0])
print("Predicted next word:", predicted_word)


Predicted next word: achineizadaepenuclide counter


## Code explanation
The code provided outlines a process of fine-tuning a language model with LoRA (Low-Rank Adaptation) by adding low-rank matrices into its attention layers, while quantizing the model for memory efficiency. Here’s a step-by-step breakdown:

### Step 1: Load and Quantize the Model
```python
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
```
1. The `model_name` variable holds the identifier for the TinyLlama model, which is loaded via Hugging Face's `AutoModelForCausalLM`.
2. The tokenizer is also loaded to convert text into token IDs for input.

```python
model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)
```
3. The `quantize_dynamic` function converts the model’s linear layers to a quantized version, reducing memory use and possibly speeding up inference. This quantization affects only the linear layers, not all components.

### Step 2: Define LoRA Matrices
```python
rank = 4
in_features = 2048
out_features = 2048
A = torch.randn(in_features, rank)  # Matrix A with shape (2048, 4)
B = torch.randn(rank, out_features) # Matrix B with shape (4, 2048)
```
4. LoRA modifies the model’s weights by introducing low-rank matrices `A` and `B` for adaptation. Here, `rank` is set to 4, meaning that each 2048-dimensional input or output will be projected into a 4-dimensional space.
5. `A` is a `2048x4` matrix (input dimension to low-rank dimension), while `B` is a `4x2048` matrix (low-rank dimension back to the original).

```python
def apply_lora(x):
    lora_output = torch.matmul(torch.matmul(x, A), B)  # shape remains (batch, seq_len, 2048)
    return lora_output
```
6. The `apply_lora` function takes in `x` (embeddings from the model) and transforms it using `A` and `B`, which adapt the model’s behavior while keeping the overall dimension unchanged.

### Step 3: Define the Forward Pass with LoRA
```python
prompt = "The cat sat on"
inputs = tokenizer(prompt, return_tensors="pt")
```
7. The input prompt is tokenized, converting it into tensor form (`inputs["input_ids"]`) for use with the model.

```python
with torch.no_grad():
    embeddings = model.model.embed_tokens(inputs["input_ids"])
```
8. Inside a `no_grad()` block (disables gradient calculations for efficiency), we get embeddings by passing the token IDs through the model’s embedding layer.

```python
attention_layer = model.model.layers[0].self_attn  # Get first attention layer
q_proj_output = attention_layer.q_proj(embeddings)
```
9. We select the first attention layer and pass the embeddings through its query projection (`q_proj`). This transformation returns `q_proj_output`, representing attention-based information of shape `(batch, seq_len, 2048)`.

```python
lora_adjustment = apply_lora(embeddings)  # Apply LoRA adjustment
attention_output = q_proj_output + lora_adjustment  # Combine with LoRA output
```
10. The LoRA transformation `apply_lora(embeddings)` is applied to the embeddings. The resulting `lora_adjustment` is added to `q_proj_output`, creating `attention_output` with the combined effects of the query projection and the LoRA adaptation.

### Step 4: Obtain Predictions
```python
logits = model.lm_head(attention_output)
```
11. The adjusted output from the attention layer is passed to the `lm_head` (language modeling head) of the model to compute `logits`, representing scores for each token in the vocabulary.

```python
probs = torch.softmax(logits, dim=-1)
predicted_token = torch.argmax(probs, dim=-1)
```
12. `softmax` is applied to `logits` to obtain probabilities for each token in the vocabulary, and `argmax` is used to select the token with the highest probability as `predicted_token`.

```python
predicted_word = tokenizer.decode(predicted_token[0])
print("Predicted next word:", predicted_word)
```
13. Finally, the predicted token ID is decoded back into a word, which is printed as the model’s next predicted word based on the prompt "The cat sat on".

### Summary
This code demonstrates the process of LoRA adaptation on an attention layer of a language model. By adding the LoRA matrices `A` and `B` to the attention layer’s query projection, it modifies the behavior of the model to specialize it for specific tasks, all while keeping model memory and computational costs lower than a full fine-tuning would require.

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define low-rank matrices A and B for LoRA with requires_grad=True for training
rank = 8  # LoRA rank, which is typically low
A = torch.randn((2048, rank), requires_grad=True)  # Low-rank projection down
B = torch.randn((rank, 2048), requires_grad=True)  # Low-rank projection up

# Function to apply LoRA transformation
def apply_lora(x):
    lora_output = x @ A @ B  # Low-rank adaptation
    return lora_output

# Fine-tuning setup: Assuming we are training LoRA parameters
optimizer = torch.optim.Adam([A, B], lr=1e-4)

# Define the number of training steps (adjust based on your needs)
num_training_steps = 100  # For example, train for 100 steps

# Define dummy target labels and loss function for demonstration purposes
# Here we’ll assume a causal language modeling task where the target is to predict the next word
loss_fn = torch.nn.CrossEntropyLoss()
target_labels = torch.tensor([1, 2, 3, 4])  # Replace with actual target labels matching your input sequence length

# Training loop for updating only LoRA matrices (A and B)
for step in range(num_training_steps):
    optimizer.zero_grad()

    # Tokenize input and pass it through the embedding layer
    inputs = tokenizer("The cat sat on", return_tensors="pt")
    embeddings = model.model.embed_tokens(inputs["input_ids"])

    # Apply LoRA to the first attention layer’s query projection
    attention_layer = model.model.layers[0].self_attn
    q_proj_output = attention_layer.q_proj(embeddings)
    lora_adjustment = apply_lora(embeddings)
    adjusted_output = q_proj_output + lora_adjustment

    # Pass through LM head to obtain logits
    logits = model.lm_head(adjusted_output)

    # Calculate loss and backpropagate to update only LoRA matrices A and B
    # Ensure target_labels has the correct shape for the loss calculation
    loss = loss_fn(logits.view(-1, logits.size(-1)), target_labels.view(-1))  # Reshape if necessary
    loss.backward()
    optimizer.step()

    print(f"Step {step + 1}/{num_training_steps}, Loss: {loss.item()}")


ValueError: Expected input batch_size (5) to match target batch_size (4).